# Introduction
This notebook shows how to setup the darknet framework, required for training the yolo object detector.
For the purpose the provided GPU environment by Google Colab will be used.
The project is using google drive to save the project files generated by the executed steps in this notebook. Of course, this is not required but is done out of convinience, so it is not required to build the project every time it is used.**bold text**

1. Import colab and drive API
2. Set workingspace directory to our google drive.

# Setting up Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Next navigate to our desired folder 
Using cd /content/gdrive/My Drive/... Folder you want.
You can make new directory by using mkdir ... - name of the folder

List the contents of our directory to make sure its what we expect

In [ ]:
!ls
%cd /content/drive/My Drive/sewer_inspection/yolo_detector
!ls

cv2.cpython-36m-x86_64-linux-gnu.so  drive  infered1.avi  sample_data
/content/drive/My Drive/sewer_inspection/yolo_detector
config_weights	cv  darknet  infered.avi  Notebooks  yolo_base	yolo_detector


## Creating a symbolic link (optional)
This step is optional, but it is recommend since it really makes working with paths in the google colab environment a lot more comfortable, especially if there are any spaces somewhere in you path.

In [ ]:
# create symbolic link for easier access to my drive
!ln -s "/content/drive/My Drive/sewer_inspection/yolo_detector/" yolo_base

ln: failed to access 'yolo_base': Operation not supported


Now every time we have to give the path to the directory where darknet for example will be installed we can use yolo_base instead.

The following following steps are to train YoloV4 model on our custom dataset.
The dataset is label with roboflow - that allows up to 500 images for free in our dataset. <br>
What we are going to try here is to use google drive as our filesystem and save and training progress to our drive, so we do not loose weights after reaching the limit for the GPU access which is 12h. <br>
[This](https://colab.research.google.com/drive/1mzL6WyY9BRx4xX476eQdhKDnd_eixBlG#scrollTo=GNVU7eu9CQj3) is the tutorial that was followed in order to realise this project.

# Configuring environment for Darknet

In [ ]:
# CUDA: Let's check that Nvidia CUDA drivers are already pre-installed and which version is it.
!/usr/local/cuda/bin/nvcc --version
# We need to install the correct cuDNN according to this output
!nvidia-smi

## Set compute capability


Change the number depending on what GPU is listed above, under NVIDIA-SMI > Name.
* Tesla K80: 30
* Tesla P100: 60
* Tesla T4: 75

In [ ]:
%env compute_capability=61

env: compute_capability=61


# Installing darknet on Colab
The following steps show how to install darknet in the working environment.
If you chose to mount google drive, the progress from the following cells can be saved and you wont have to repeat those steps again.

The cell below will remove darknet directory if there is one and clone the Darknet repository.

In [ ]:
!ls

3rdparty	darknet			include		       README.md
backup		DarknetConfig.cmake.in	json_mjpeg_streams.sh  results
build		darknet_images.py	LICENSE		       scripts
build.ps1	darknet.py		Makefile	       src
build.sh	darknet_video.py	net_cam_v3.sh	       video_yolov3.sh
cfg		data			net_cam_v4.sh	       video_yolov4.sh
cmake		image_yolov3.sh		obj		       yolov4.weights
CMakeLists.txt	image_yolov4.sh		predictions.jpg


In [ ]:
%rm -rf darknet
#we clone the fork of darknet maintained by roboflow
#small changes have been made to configure darknet for training
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14370, done.
remote: Total 14370 (delta 0), reused 0 (delta 0), pack-reused 14370
Receiving objects: 100% (14370/14370), 13.09 MiB | 22.88 MiB/s, done.
^C


In [ ]:
#install environment from the Makefile
#note if you are on Colab Pro this works on a P100 GPU
#if you are on Colab free, you may need to change the Makefile for the K80 GPU
#this goes for any GPU, you need to change the Makefile to inform darknet which GPU you are running on.
%cd darknet/
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i "s/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= -gencode arch=compute_${compute_capability},code=sm_${compute_capability}/g" Makefile
!make

#Test pre-trained yolo detector

In the next cell the weights for yolo4 trained on coco dataset are downloaded and tested

The next cell downloads the config and weights for yolo trained on coco dataset.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

The following cells contains some functionts that will help us show the results from running the detection.

## Testing yolo 4 trained on coco dataset

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

The cell below, once executed will run darknet with the yolo4 detector trained on the coco dataset.

In [ ]:
!./darknet detect cfg/yolov4.cfg yolov4.weights data/dog.jpg
imShow('predictions.jpg')

# Training custom yolo 4 detector
The following cells when after executed will create custom yolo 4 configuration that can be trained on our own dataset.

## Download dataset
Using roboflow to create dataset. Download using the link provided by roboflow

In [ ]:
#if you already have YOLO darknet format, you can skip this step
%cd yolo_base/darknet
#!curl -L https://app.roboflow.com/ds/NH2ig2l5j5?key=S4Qkqs3roo > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
#Set up training file directories for custom dataset
%cd yolo_base/darknet #%cd /content/gdrive/My Drive/NNTraining/darknet
%cp train/_darknet.labels data/obj.names
%mkdir data/obj
#copy image and labels
%cp train/*.jpg data/obj/
%cp valid/*.jpg data/obj/

%cp train/*.txt data/obj/
%cp valid/*.txt data/obj/

with open('data/obj.data', 'w') as out:
  out.write('classes = 3\n')
  out.write('train = data/train.txt\n')
  out.write('valid = data/valid.txt\n')
  out.write('names = data/obj.names\n')
  out.write('backup = backup/')

#write train file (just the image list)
import os

with open('data/train.txt', 'w') as out:
  for img in [f for f in os.listdir('train') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')

#write the valid file (just the image list)
import os

with open('data/valid.txt', 'w') as out:
  for img in [f for f in os.listdir('valid') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')

In [ ]:
#we build config dynamically based on number of classes
#we build iteratively from base config files. This is the same file shape as cfg/yolo-obj.cfg
def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = file_len('train/_darknet.labels')
print("writing config for a custom YOLOv4 detector detecting number of classes: " + str(num_classes))

#Instructions from the darknet repo
#change line max_batches to (classes*2000 but not less than number of training images, and not less than 6000), f.e. max_batches=6000 if you train for 3 classes
#change line steps to 80% and 90% of max_batches, f.e. steps=4800,5400
if os.path.exists('./cfg/custom-yolov4-detector.cfg'): os.remove('./cfg/custom-yolov4-detector.cfg')


with open('./cfg/custom-yolov4-detector.cfg', 'a') as f:
  f.write('[net]' + '\n')
  f.write('batch=64' + '\n')
  #####smaller subdivisions help the GPU run faster. 12 is optimal, but you might need to change to 24,36,64####
  f.write('subdivisions=24' + '\n')
  f.write('width=416' + '\n')
  f.write('height=416' + '\n')
  f.write('channels=3' + '\n')
  f.write('momentum=0.949' + '\n')
  f.write('decay=0.0005' + '\n')
  f.write('angle=0' + '\n')
  f.write('saturation = 1.5' + '\n')
  f.write('exposure = 1.5' + '\n')
  f.write('hue = .1' + '\n')
  f.write('\n')
  f.write('learning_rate=0.001' + '\n')
  f.write('burn_in=1000' + '\n')
  ######you can adjust up and down to change training time#####
  ##Darknet does iterations with batches, not epochs####
  max_batches = num_classes*2000
  #max_batches = 2000
  f.write('max_batches=' + str(max_batches) + '\n')
  f.write('policy=steps' + '\n')
  steps1 = .8 * max_batches
  steps2 = .9 * max_batches
  f.write('steps='+str(steps1)+','+str(steps2) + '\n')

#Instructions from the darknet repo
#change line classes=80 to your number of objects in each of 3 [yolo]-layers:
#change [filters=255] to filters=(classes + 5)x3 in the 3 [convolutional] before each [yolo] layer, keep in mind that it only has to be the last [convolutional] before each of the [yolo] layers.

  with open('cfg/yolov4-custom2.cfg', 'r') as f2:
    content = f2.readlines()
    for line in content:
      f.write(line)    
    num_filters = (num_classes + 5) * 3
    f.write('filters='+str(num_filters) + '\n')
    f.write('activation=linear')
    f.write('\n')
    f.write('\n')
    f.write('[yolo]' + '\n')
    f.write('mask = 0,1,2' + '\n')
    f.write('anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401' + '\n')
    f.write('classes=' + str(num_classes) + '\n')

  with open('cfg/yolov4-custom3.cfg', 'r') as f3:
    content = f3.readlines()
    for line in content:
      f.write(line)    
    num_filters = (num_classes + 5) * 3
    f.write('filters='+str(num_filters) + '\n')
    f.write('activation=linear')
    f.write('\n')
    f.write('\n')
    f.write('[yolo]' + '\n')
    f.write('mask = 3,4,5' + '\n')
    f.write('anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401' + '\n')
    f.write('classes=' + str(num_classes) + '\n')

  with open('cfg/yolov4-custom4.cfg', 'r') as f4:
    content = f4.readlines()
    for line in content:
      f.write(line)    
    num_filters = (num_classes + 5) * 3
    f.write('filters='+str(num_filters) + '\n')
    f.write('activation=linear')
    f.write('\n')
    f.write('\n')
    f.write('[yolo]' + '\n')
    f.write('mask = 6,7,8' + '\n')
    f.write('anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401' + '\n')
    f.write('classes=' + str(num_classes) + '\n')
    
  with open('cfg/yolov4-custom5.cfg', 'r') as f5:
    content = f5.readlines()
    for line in content:
      f.write(line)

print("file is written!")    

writing config for a custom YOLOv4 detector detecting number of classes: 5
file is written!


In [ ]:
#here is the file that was just written. 
#you may consider adjusting certain things

#like the number of subdivisions 64 runs faster but Colab GPU may not be big enough
#if Colab GPU memory is too small, you will need to adjust subdivisions to 16
%cat cfg/custom-yolov4-detector.cfg

In [ ]:
%cd /yolo_base/darknet
!ls

In [ ]:
#If you get CUDA out of memory adjust subdivisions above!
#adjust max batches down for shorter training above
!./darknet detector train data/obj.data cfg/custom-yolov4-detector.cfg yolov4.conv.137 -dont_show -map

Executing this cell will start training the neural network from the last known point.

In [ ]:
!./darknet detector train data/obj.data cfg/custom-yolov4-detector.cfg backup/custom-yolov4-detector_last.weights -dont_show -map

# Infer custom objects with saved weights

In [ ]:
#check if weigths have saved yet
#backup houses the last weights for our detector
#(file yolo-obj_last.weights will be saved to the build\darknet\x64\backup\ for each 100 iterations)
#(file yolo-obj_xxxx.weights will be saved to the build\darknet\x64\backup\ for each 1000 iterations)
#After training is complete - get result yolo-obj_final.weights from path build\darknet\x64\bac
!ls backup
#if it is empty you haven't trained for long enough yet, you need to train for at least 100 iterations

In [ ]:
import os

#define utility function
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

#/test has images that we can test our detector on
#test_images = [f for f in os.listdir('test') if f.endswith('.jpg')]
import random
#img_path = "test/" + random.choice(test_images);
# can use {img_path} instead of path, if test folder i available
#Yolo 4 config
! ./darknet detector test \
cfg/coco.data \
../../../config_weights/yolo_minor/yolov3_custom_C4_C3_C4D_C3D.cfg \
../../../config_weights/yolo_minor/yolov3_custom_final.weights \
 ../config_weights/yolo_minor/test.jpg
#Yolo 3 minor group config
#!./darknet detector test cfg/coco.data ../config_weights/yolo_minor/yolov3_custom_C4_C3_C4D_C3D.cfg ../config_weights/yolo_minor/yolov3_custom_final.weights ../config_weights/yolo_minor/test.jpg 
imShow('predictions.jpg')

# Inferencing Yolo detector with OpenCV DNN module



In [ ]:
!cp -r "/content/drive/My Drive/sewer_inspection/opencv_gpu/cv2.cpython-36m-x86_64-linux-gnu.so" .

In [ ]:
# import the necessary packages
import numpy as np
import time
import cv2
from google.colab.patches import cv2_imshow 

cv2.__version__

'4.5.0-dev'

In [ ]:
#Yolo4
labelsPath = '/content/drive/My Drive/sewer_inspection/yolo_detector/config_weights/yolo4/obj.names'  # damages label path
weightsPath = '/content/drive/My Drive/sewer_inspection/yolo_detector/config_weights/yolo4/custom-yolov4-detector_best.weights'  # damages paths
configPath = '/content/drive/My Drive/sewer_inspection/yolo_detector/config_weights/yolo4/yolov4-custom.cfg'

In [ ]:
#Yolo3 minor
labelsPath='/content/drive/My Drive/sewer_inspection/yolo_detector/config_weights/yolo_minor/obj.names'
weightsPath = '/content/drive/My Drive/sewer_inspection/yolo_detector/config_weights/yolo_minor/yolov3_custom_C3_C4_C3D_C4D_I.weights'  # damages paths
configPath = '/content/drive/My Drive/sewer_inspection/yolo_detector/config_weights/yolo_minor/yolov3_custom_C4_C3_C4D_C3D_I.cfg'

In [ ]:
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
LABELS = open(labelsPath).read().strip().split("\n")

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

[INFO] loading YOLO from disk...


In [ ]:
# NN params
confidence_param = 0.30
thresh_param = 0.25

### Test with image

In [ ]:
def detect_from_image(frame, verbose = False):
  try:            
      (H, W) = frame.shape[:2]
      blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),	swapRB=False, crop=False)
      net.setInput(blob)
      start = time.time()
      layerOutputs = net.forward(ln)
      end = time.time()
      elap = (end - start)
      print("[INFO] single frame took {:.4f} seconds".format(elap))
      # initialize our lists of detected bounding boxes, confidences,
      # and class IDs, respectively
      boxes = []
      confidences = []
      classIDs = []
  # loop over each of the layer outputs
      for output in layerOutputs:
          # loop over each of the detections
          for detection in output:
              # extract the class ID and confidence (i.e., probability)
              # of the current object detection
              scores = detection[5:]
              classID = np.argmax(scores)
              confidence = scores[classID]

              # filter out weak predictions by ensuring the detected
              # probability is greater than the minimum probability
              if confidence > confidence_param:
                  # scale the bounding box coordinates back relative to
                  # the size of the image, keeping in mind that YOLO
                  # actually returns the center (x, y)-coordinates of
                  # the bounding box followed by the boxes' width and
                  # height
                  box = detection[0:4] * np.array([W, H, W, H])
                  (centerX, centerY, width, height) = box.astype("int")

                  # use the center (x, y)-coordinates to derive the top
                  # and and left corner of the bounding box
                  x = int(centerX - (width / 2))
                  y = int(centerY - (height / 2))

                  # update our list of bounding box coordinates,
                  # confidences, and class IDs
                  boxes.append([x, y, int(width), int(height)])
                  confidences.append(float(confidence))
                  classIDs.append(classID)
                  if verbose :
                    print("Score:{} ClassId:{} Confidence:{} boxes:{}".format(scores,classID, confidence, [x, y, int(width), int(height)]))


  # apply non-maxima suppression to suppress weak, overlapping
  # bounding boxes
      idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence_param,
                              thresh_param)

      # ensure at least one detection exists
      if len(idxs) > 0:
          # loop over the indexes we are keeping
          for i in idxs.flatten():
              # extract the bounding box coordinates
              (x, y) = (boxes[i][0], boxes[i][1])
              (w, h) = (boxes[i][2], boxes[i][3])

              # draw a bounding box rectangle and label on the frame
              color = [int(c) for c in COLORS[classIDs[i]]]
              cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
              text = "{}: {:.4f}".format(LABELS[classIDs[i]],
                                        confidences[i])
              cv2.putText(frame, text, (x, y - 5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)       
            # some information on processing single frame
  except Exception as e:
    print("something happened: ", e)
      
  return frame

In [ ]:
# read input image
image_path='/content/drive/My Drive/sewer_inspection/yolo_detector/config_weights/yolo_minor/1.png'
image = cv2.imread(image_path)
# process image
processed = detect_from_image(image, True)
# display
cv2_imshow(image)

In [ ]:
print("reading video")
cam = cv2.VideoCapture('/content/drive/My Drive/sewer_inspection/yolo_detector/config_weights/high_quality.mpg')
ret, frame = cam.read()

while cam.isOpened():
    ret, frame = cam.read()
    frame = detect_from_image(frame, False)
    cv2_imshow(frame)



In [ ]:
print("reading video")
cam = cv2.VideoCapture('/content/drive/My Drive/sewer_inspection/yolo_detector/config_weights/high_quality.mpg')
ret, frame = cam.read()

height, width, layers = frame.shape


name = '/content/drive/My Drive/sewer_inspection/yolo_detector/infered.avi'
out = cv2.VideoWriter(name, cv2.VideoWriter_fourcc(*'DIVX'), 25, (width, height))
counter1000frames = 0
while cam.isOpened():
    ret, frame = cam.read()
    frame = detect_from_image(frame, False)
    out.write(frame)

out.release()

print("saved {}" ,name)